<a href="https://colab.research.google.com/github/edmand88/math-problem-classifier/blob/main/math_problems_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat May 18 10:54:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   31C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import pickle

import sklearn.preprocessing as skprep

tf.random.set_seed(42)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pylatexenc.latex2text import LatexNodes2Text

def latex_converter(latex_str):
    converter = LatexNodes2Text()
    return converter.latex_to_text(latex_str)

def remove_newlines(text):
    return text.replace('\n', '')

def remove_literal_newlines(text):
    return text.replace('\\n', '')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data/external_df.csv")
df['problem'] = df['problem'].apply(latex_converter)
df['problem'] = df['problem'].apply(remove_newlines)
df['problem'] = df['problem'].apply(remove_literal_newlines)
df['problem'] = df['problem'].str.replace(r'\s+', ' ', regex=True).str.strip()

math_df = df[df['source']=='MATH']
math_df = math_df.drop('source', axis=1)
math_df = math_df.drop('level', axis=1)
math_df = math_df.drop('solution', axis=1)

math_df['type'] = math_df['type'].apply(lambda x: {'Intermediate Algebra': 'Algebra', 'Prealgebra': 'Algebra'}.get(x, x))

print(math_df)

                                                 problem         type  stage
0      Kevin Kangaroo begins hopping on a number line...      Algebra  train
1      The ratio of the areas of two squares is 192/8...      Algebra  train
2                 If √(2√(t-2)) = √(7 - t), then find t.      Algebra  train
3      Let t(x) = √(3x+1) and f(x)=5-t(x). What is t(...      Algebra  train
4      James has a total of 66 dollars in his piggy b...      Algebra  train
...                                                  ...          ...    ...
12495  Find the matrix 𝐋 = [ a 0; c d ] such that a >...  Precalculus   test
12496  Find all values of x, 0 ≤ x ≤ 2 π, which satis...  Precalculus   test
12497  Let 𝐚, 𝐛, 𝐜 be three vectors such that 𝐚×𝐛 = [...  Precalculus   test
12498  Let x, y, and z be real numbers such that cos ...  Precalculus   test
12499  Below is the graph of y = a bx for some positi...  Precalculus   test

[12500 rows x 3 columns]


In [ ]:
math_df.groupby('stage').describe()

problem                                                                 \
        count unique                                                top freq   
stage                                                                          
test     5000   5000  Below is the graph of y = a bx for some positi...    1   
train    7500   7499  Real numbers x and y satisfy the equation x^2 ...    2   

       type                        
      count unique      top  freq  
stage                              
test   5000      5  Algebra  2961  
train  7500      5  Algebra  4244

In [ ]:
df_training_type = math_df[math_df['stage']=='train']
df_training_type.groupby('type').describe()

problem         \
                         count unique   
type                                    
Algebra                   4244   4243   
Counting & Probability     771    771   
Geometry                   870    870   
Number Theory              869    869   
Precalculus                746    746   

                                                                           \
                                                                      top   
type                                                                        
Algebra                 Real numbers x and y satisfy the equation x^2 ...   
Counting & Probability  A bin has 8 black balls and 7 white balls. 3 o...   
Geometry                Two chords, AB and CD, meet inside a circle at...   
Number Theory           What is the smallest positive integer that has...   
Precalculus             When the vectors [ 4; 1 ] and [ -1; 3 ] are bo...   

                            stage                      
                       freq count unique    top  freq  
type                                                   
Algebra                   2  4244      1  train  4244  
Counting & Probability    1   771      1  train   771  
Geometry                  1   870      1  train   870  
Number Theory             1   869      1  train   869  
Precalculus               1   746      1  train   746

In [ ]:
df_type1 = df_training_type[df_training_type['type']=='Algebra']
df_type2 = df_training_type[df_training_type['type']=='Counting & Probability']
df_type3 = df_training_type[df_training_type['type']=='Geometry']
df_type4 = df_training_type[df_training_type['type']=='Number Theory']
df_type5 = df_training_type[df_training_type['type']=='Precalculus']

df_type1_downsampled = df_type1.sample(746)
df_type2_downsampled = df_type2.sample(746)
df_type3_downsampled = df_type3.sample(746)
df_type4_downsampled = df_type4.sample(746)
df_type5_downsampled = df_type5.sample(746)

df_training_type_balanced = pd.concat([df_type1_downsampled, df_type2_downsampled, df_type3_downsampled, df_type4_downsampled, df_type5_downsampled])

In [ ]:
X_train, y_train = df_training_type_balanced['problem'], df_training_type_balanced['type']

In [ ]:
df_test_type = math_df[math_df['stage']=='test']
df_test_type.groupby('type').describe()

problem         \
                         count unique   
type                                    
Algebra                   2961   2961   
Counting & Probability     474    474   
Geometry                   479    479   
Number Theory              540    540   
Precalculus                546    546   

                                                                           \
                                                                      top   
type                                                                        
Algebra                 How many degrees are in the acute angle formed...   
Counting & Probability                      What is the value of (3!)!3!?   
Geometry                Sally has a cube of side length s units such t...   
Number Theory           Add 1_2 + 11_2 + 111_2 + 1111_2. Express your ...   
Precalculus             Below is the graph of y = a bx for some positi...   

                            stage                     
                       freq count unique   top  freq  
type                                                  
Algebra                   1  2961      1  test  2961  
Counting & Probability    1   474      1  test   474  
Geometry                  1   479      1  test   479  
Number Theory             1   540      1  test   540  
Precalculus               1   546      1  test   546

In [ ]:
df_val_type = df_test_type.sample(500)

In [ ]:
X_val, y_val = df_val_type['problem'], df_val_type['type']
X_test, y_test = df_test_type['problem'], df_test_type['type']

In [ ]:
tokenizer3 = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
tokenizer3.fit_on_texts(X_train)
X_train_seq3 = tokenizer3.texts_to_sequences(X_train)
X_test_seq3 = tokenizer3.texts_to_sequences(X_test)

max_length = 833
X_train_pad3 = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq3, maxlen=max_length)
X_test_pad3 = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq3, maxlen=max_length)

encoder3 = skprep.LabelEncoder()
y_train_encoded3 = encoder3.fit_transform(y_train)
y_test_encoded3 = encoder3.transform(y_test)
y_train_cat3 = tf.keras.utils.to_categorical(y_train_encoded3)
y_test_cat3 = tf.keras.utils.to_categorical(y_test_encoded3)

In [ ]:
modelX = tf.keras.models.Sequential()
modelX.add(tf.keras.layers.Embedding(input_dim=len(tokenizer3.word_index) + 1, output_dim=50, input_length=max_length, mask_zero=True))
modelX.add(tf.keras.layers.Dropout(rate=0.2))
modelX.add(tf.keras.layers.GRU(150, dropout=0.5))
modelX.add(tf.keras.layers.Dropout(rate=0.2))
modelX.add(tf.keras.layers.Dense(5, activation='softmax'))

In [ ]:
modelX.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint_cbX = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Data/my_checkpointsX", save_best_only=True)

modelX.fit(X_train_pad3, y_train_cat3, epochs=30, batch_size=32, validation_data=(X_test_pad3, y_test_cat3), callbacks=[checkpoint_cbX])

Epoch 1/30
117/117 [==============================] - 195s 2s/step - loss: 1.3550 - accuracy: 0.4434 - val_loss: 1.2289 - val_accuracy: 0.4534
Epoch 2/30
117/117 [==============================] - 181s 2s/step - loss: 0.8690 - accuracy: 0.6748 - val_loss: 0.9306 - val_accuracy: 0.6670
Epoch 3/30
117/117 [==============================] - 178s 2s/step - loss: 0.6943 - accuracy: 0.7590 - val_loss: 0.8121 - val_accuracy: 0.7016
Epoch 4/30
117/117 [==============================] - 169s 1s/step - loss: 0.6593 - accuracy: 0.7769 - val_loss: 0.8482 - val_accuracy: 0.6672
Epoch 5/30
117/117 [==============================] - 176s 2s/step - loss: 0.5806 - accuracy: 0.8003 - val_loss: 0.8071 - val_accuracy: 0.7068
Epoch 6/30
117/117 [==============================] - 169s 1s/step - loss: 0.4901 - accuracy: 0.8322 - val_loss: 0.8208 - val_accuracy: 0.6992
Epoch 7/30
117/117 [==============================] - 174s 1s/step - loss: 0.4602 - accuracy: 0.8362 - val_loss: 0.7535 - val_accuracy: 0.7096

In [3]:
modelX = tf.keras.models.load_model('/content/drive/MyDrive/Data/my_checkpointsX')

In [4]:
with open('/content/drive/MyDrive/Data/tokenizer3.pkl', 'rb') as file:
    loaded_tokenizer3 = pickle.load(file)

with open('/content/drive/MyDrive/Data/encoder3.pkl', 'rb') as file:
    loaded_encoder3 = pickle.load(file)

In [5]:
X_train_pad3_loaded = np.load('/content/drive/MyDrive/Data/X_train_pad3.npy')
X_test_pad3_loaded = np.load('/content/drive/MyDrive/Data/X_test_pad3.npy')
y_train_encoded3_loaded = np.load('/content/drive/MyDrive/Data/y_train_encoded3.npy')
y_test_encoded3_loaded = np.load('/content/drive/MyDrive/Data/y_test_encoded3.npy')
y_train_cat3_loaded = np.load('/content/drive/MyDrive/Data/y_train_cat3.npy')
y_test_cat3_loaded = np.load('/content/drive/MyDrive/Data/y_test_cat3.npy')

In [ ]:
lossX, accuracyX = modelX.evaluate(X_test_pad3_loaded, y_test_cat3_loaded)
print("Test Loss:", lossX)
print("Test Accuracy:", accuracyX)

157/157 [==============================] - 25s 162ms/step - loss: 0.7090 - accuracy: 0.7522
Test Loss: 0.7089696526527405
Test Accuracy: 0.7522000074386597


In [36]:
new_text = ["How many unique pairs of students can be formed from a group of 8 students?"]

new_text_seq = loaded_tokenizer3.texts_to_sequences(new_text)

new_text_pad = tf.keras.preprocessing.sequence.pad_sequences(new_text_seq, maxlen=833)

prediction = modelX.predict(new_text_pad)

predicted_class_index = np.argmax(prediction, axis=1)
predicted_class = loaded_encoder3.inverse_transform(predicted_class_index)
print("Predicted class name:", predicted_class)

1/1 [==============================] - 0s 72ms/step
Predicted class name: ['Counting & Probability']
